In [83]:
import pandas as pd
import sklearn, sklearn.tree, numpy
import graphviz

In [4]:
!ls data

amazon_baby_subset.gl important_words.json  lending-club-data.csv


In [7]:
loandata = pd.read_csv('data/lending-club-data.csv', dtype={'url':'string'})

In [17]:
print [n for n in loandata.columns.values]

['id', 'member_id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'is_inc_v', 'issue_d', 'loan_status', 'pymnt_plan', 'url', 'desc', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt', 'next_pymnt_d', 'last_credit_pull_d', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'not_compliant', 'status', 'inactive_loans', 'bad_loans', 'emp_length_num', 'grade_num', 'sub_grade_num', 'delinq_2yrs_zero', 'pub_rec_zero', 'collections_12_mths_zero', 'short_emp', 'payment_in

In [21]:
loandata['safe_loan'] = loandata['bad_loans'].apply(lambda x: +1 if x==0 else -1)

In [24]:
ok = loandata[(loandata.safe_loan ==1)]

In [25]:
len(ok)

99457

In [26]:
bad = loandata[(loandata.safe_loan==-1)]

In [27]:
len(bad)

23150

In [28]:
print len(ok)/(len(bad) +len(ok))

0


In [33]:
print float(len(ok))/(float(len(ok))+float(len(bad)))

0.811185331996


**81.1% safe loans, 19% bad loan**

In [41]:
features = ['grade',                     # grade of the loan
            'sub_grade',                 # sub-grade of the loan
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'term',                      # the term of the loan
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
           ]
target = 'safe_loan'

In [70]:
#we extract the feature columns and target column
loansdata = loandata[features + [target]]
loansdata.head()

,grade,sub_grade,short_emp,emp_length_num,home_ownership,dti,purpose,term,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,safe_loan
0,B,B2,0,11,RENT,27.65,credit_card,36 months,1,1,83.7,0.00,1
1,C,C4,1,1,RENT,1.00,car,60 months,1,1,9.4,0.00,-1
2,C,C5,0,11,RENT,8.72,small_business,36 months,1,1,98.5,0.00,1
3,C,C1,0,11,RENT,20.00,other,36 months,0,1,21.0,16.97,1
4,A,A4,0,4,RENT,11.20,wedding,36 months,1,1,28.3,0.00,1


In [71]:
# one hot encoding converts the data aset into machine splittable version so we are not deailing
# with strings
loans_ohe = pd.get_dummies(loansdata)
loans_ohe.head()

,short_emp,emp_length_num,dti,last_delinq_none,last_major_derog_none,revol_util,total_rec_late_fee,safe_loan,grade_A,grade_B,...,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding,term_ 36 months,term_ 60 months
0,0,11,27.65,1,1,83.7,0.00,1,0,1,...,0,0,0,0,0,0,0,0,1,0
1,1,1,1.00,1,1,9.4,0.00,-1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,0,11,8.72,1,1,98.5,0.00,1,0,0,...,0,0,0,0,0,1,0,0,1,0
3,0,11,20.00,0,1,21.0,16.97,1,0,0,...,0,0,0,0,1,0,0,0,1,0
4,0,4,11.20,1,1,28.3,0.00,1,1,0,...,0,0,0,0,0,0,0,1,1,0


In [85]:
#load the train and validation idx files downloaded
import json

with open('data/module-5-assignment-1-train-idx.json', 'r') as f: # Reads the list of most frequent words
    train_idx = json.load(f)
f.close()

with open('data/module-5-assignment-1-validation-idx.json','r') as f:
    validation_idx = json.load(f)
f.close()
    #important_words = [str(s) for s in important_words]
#train_idx = f

In [86]:
# select the specific samples from the training and validation set in the loans one hot encoded data
train_data = loans_ohe.iloc[train_idx]
validation_data= loans_ohe.iloc[validation_idx]

In [123]:
train_data['safe_loan'].head()

1    -1
6    -1
7    -1
10   -1
12   -1
Name: safe_loan, dtype: int64

In [61]:
train_array = train_data.as_matrix()

In [184]:
#the decision tree
dtree = sklearn.tree.DecisionTreeClassifier(max_depth=6)
dtree_y = train_data['safe_loan']
dtree_X = train_data.drop('safe_loan',1)
decision_tree_model = dtree.fit(dtree_X,dtree_y)
small_model = sklearn.tree.DecisionTreeClassifier(max_depth=2).fit(dtree_X,dtree_y)


In [185]:
#make predictions
validation_y = validation_data['safe_loan']
validation_X = validation_data.drop('safe_loan',1)
validation_safe_loans = validation_data[validation_data[target] ==1]
validation_risky_loans = validation_data[validation_data[target] == -1]
sample_validation_data_risky = validation_risky_loans[0:2]
sample_validation_data_safe = validation_safe_loans[0:2]
sample_validation_data = sample_validation_data_safe.append(sample_validation_data_risky)
sample_validation_data = sample_validation_data.drop(target,1)
print sample_validation_data.shape

(4, 67)


In [186]:
decision_tree_model.predict(sample_validation_data)


array([ 1, -1, -1,  1])

**50% accuracy**

In [187]:
decision_tree_model.predict_proba(sample_validation_data)

array([[ 0.34156543,  0.65843457],
       [ 0.53630646,  0.46369354],
       [ 0.64750958,  0.35249042],
       [ 0.20789474,  0.79210526]])

In [188]:
small_model.predict(sample_validation_data)

array([ 1, -1, -1,  1])

In [189]:
small_model.predict_proba(sample_validation_data)

array([[ 0.41896585,  0.58103415],
       [ 0.59255339,  0.40744661],
       [ 0.59255339,  0.40744661],
       [ 0.23120112,  0.76879888]])

In [190]:
print small_model.score(dtree_X, dtree_y)
print decision_tree_model.score(dtree_X,dtree_y)

0.613502041694
0.640527616591


In [192]:
small_model.predict(sample_validation_data.iloc[[1]])

array([-1])

In [153]:
small_model.score(validation_X,validation_y)

0.61934510986643687

In [154]:

big_model = sklearn.tree.DecisionTreeClassifier(max_depth=10).fit(dtree_X,dtree_y)


In [202]:
print decision_tree_model.score(dtree_X,dtree_y)
print big_model.score(dtree_X,dtree_y)

0.640527616591
0.66379217709


In [204]:
print decision_tree_model.score(validation_X,validation_y)
print big_model.score(validation_X, validation_y)

0.636148211978
0.626561826799


In [227]:
out = decision_tree_model.predict(validation_X)
print validation_y.iloc[[0]].values[0]
#for x in numpy.nditer(out):
#    print x

-1


In [230]:
len(out)

9284

In [232]:
correct_count = false_positive = false_negative = 0

for i in range(len(out)):
    print "i = %d val = %d out = %d" % (i,validation_y.iloc[[i]].values[0], out[i])
    if validation_y.iloc[[i]].values[0] == out[i]:
        correct_count = correct_count+1
    elif out[i] == 1:
        false_negative = false_negative+1
    elif out[i] == -1:
        print out[i]
        false_positive = false_positive+1
        
print " correct %d" % correct_count
print " false positive %d" % false_positive
print " false negative %d" % false_negative

i = 0 val = -1 out = -1
i = 1 val = -1 out = 1
i = 2 val = -1 out = -1
correct
i = 3 val = -1 out = -1
i = 4 val = -1 out = 1
i = 5 val = -1 out = -1
i = 6 val = -1 out = 1
i = 7 val = -1 out = 1
i = 8 val = -1 out = -1
i = 9 val = -1 out = -1
i = 10 val = -1 out = -1
i = 11 val = -1 out = 1
i = 12 val = -1 out = 1
i = 13 val = -1 out = -1
i = 14 val = -1 out = 1
i = 15 val = -1 out = 1
i = 16 val = -1 out = 1
i = 17 val = -1 out = -1
i = 18 val = -1 out = -1
i = 19 val = -1 out = -1
i = 20 val = -1 out = 1
i = 21 val = -1 out = 1
i = 22 val = -1 out = -1
i = 23 val = -1 out = -1
i = 24 val = -1 out = -1
i = 25 val = -1 out = 1
i = 26 val = -1 out = -1
i = 27 val = -1 out = -1
i = 28 val = -1 out = 1
i = 29 val = -1 out = -1
i = 30 val = -1 out = -1
i = 31 val = -1 out = -1
i = 32 val = -1 out = -1
i = 33 val = -1 out = 1
i = 34 val = -1 out = 1
i = 35 val = -1 out = -1
i = 36 val = -1 out = 1
i = 37 val = -1 out = -1
i = 38 val = -1 out = 1
i = 39 val = -1 out = -1
i = 40 val = -1 out

In [180]:
mistakes = false_negative + false_positive

In [233]:
10000*false_negative+20000*false_positive

50950000